<a href="https://colab.research.google.com/github/FinCode-17/Options-Futures/blob/main/Jump_Diffusion_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.stats as ss
import time
from scipy.optimize import root


def BS(S0,K,T,r,sigma,call = 1,QuoteCon = 0):
    if QuoteCon > 0:
        r = QuoteCon*np.log(1+r/QuoteCon)
    d2 = (np.log(S0/K)+(r-sigma**2/2)*T)/(sigma*np.sqrt(T))
    d1 = d2+sigma*np.sqrt(T)
    if call == 1:
        return S0*ss.norm.cdf(d1)-K*np.exp(-r*T)*ss.norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*ss.norm.cdf(-d2)-S0*ss.norm.cdf(-d1)


def IVOL(S0,K,T,r,c_K,call = 1,QuoteCon = 0):
    if QuoteCon > 0:
        r = QuoteCon*np.log(1+r/QuoteCon)
    def temp(v):
        return BS(S0,K,T,r,v,call,QuoteCon)-c_K
    res = root(temp,[2])
    return res.x[0]

In [ ]:
def STxNwJumps(S0,r,sigma,crash_rate,mean_crash_loss,crash_sigma,T,N):

    drift = r-(sigma**2)/2+mean_crash_loss*crash_rate
    normal_ret = np.random.normal(drift*T,sigma*np.sqrt(T),N);
    jumps = np.random.poisson(crash_rate*T,N)
    def jumpret(j):
        return (j*np.log(1-mean_crash_loss)+
                np.random.normal(-j*crash_sigma**2/2,np.sqrt(j)*crash_sigma))
    jump_ret = jumpret(jumps)
    ST = S0*np.exp(normal_ret+jump_ret)
    return ST

In [ ]:
S0=100;
r=0.02; sigma=0.5
crash_rate=1/10; mean_crash_loss=0.20; crash_sigma=0.3
T=1;

N=10**6

t = time.time()
print(np.exp(-r*T)*np.mean(STxNwJumps(S0,r,sigma,crash_rate,mean_crash_loss,crash_sigma,T,N)))
print(time.time()-t)

In [ ]:
def JumpDiffMCCall(S0,K,r,T,sigma,crash_rate,mean_crash_loss,crash_sigma,N,call = 1):
    ST = STxNwJumps(S0,r,sigma,crash_rate,mean_crash_loss,crash_sigma,T,N)
    if call == 1:
        payoff = ST-K
    else:
        payoff = K-ST
    payoff[payoff<0] = 0
    return np.exp(-r*T)*np.mean(payoff)

In [ ]:
S0=100; K=50; r=0.02; T=1
sigma=0.5; crash_rate=1/10; mean_crash_loss=0.20
crash_sigma=0.3
N=10**7

print(BS(S0,K,T,r,sigma,0))
print(JumpDiffMCCall(S0,K,r,T,sigma,0,mean_crash_loss,crash_sigma,N,0))
print(JumpDiffMCCall(S0,K,r,T,sigma,crash_rate,mean_crash_loss,crash_sigma,N,0))

In [ ]:
import matplotlib.pyplot as plt

S0=100; r=0.02;
sigma=0.16
crash_rate=1/10;
mean_crash_loss=0.20;
crash_sigma=0.30

N=10**7

TTM = [1,1/4,1/12];

Kmin = 80
Kmax = 130
stepsize = 1;
strikerange = [Kmin,Kmax+stepsize];
KN = round((strikerange[1]-strikerange[0])/stepsize);

JD_IVOL = np.zeros((KN,len(TTM)))
tempt = time.time()
for k in range(KN):
    K = strikerange[0]+k*stepsize
    if K%10 == 0:
        print([K, time.time()-tempt])
        tempt = time.time()
    for t in range(len(TTM)):
        T = TTM[t]
        CP = JumpDiffMCCall(S0,K,r,T,sigma,crash_rate,mean_crash_loss,crash_sigma,N)
        IV = IVOL(S0,K,T,r,CP)
        JD_IVOL[k,t] = IV

strikes = np.arange(strikerange[0],strikerange[1],stepsize)
plt.plot(strikes, JD_IVOL)

In [ ]:
N=10**6

S0=100;
K_ATM=S0;
K=80;
r=0.05;
T = 1/4;
sigma=0.16
crash_rate=1/10;
mean_crash_loss=0.20;
crash_sigma=0.30

temp = time.time()

CP = JumpDiffMCCall(S0,K_ATM,r,T,sigma,crash_rate,mean_crash_loss,crash_sigma,N)
IV_ATM = IVOL(S0,K_ATM,T,r,CP)
CP = JumpDiffMCCall(S0,K,r,T,sigma,crash_rate,mean_crash_loss,crash_sigma,N)
IV = IVOL(S0,K,T,r,CP)
print(100*IV_ATM)
print(100*IV)
print(' ')
print(time.time()-temp)